<a href="https://colab.research.google.com/github/frikel12/Machine-Learning-and-Deep-Learningproject/blob/main/Moroccan_Sentiment_Analysis_using_a_fine_tune_bert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install accelerate==0.20.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
import torch
import pickle


# <b>Load Data

In [ ]:
path='/content/drive/My Drive/Datasets/marocSA/train.csv'

train = pd.read_csv(path)
train.head()

,ID,comment,label
0,1,انا اوصي من هذا المنبر الكل للتوجه الى مراكز ا...,1
1,2,\nهناك الكثير لا يفهم المقصود كورونا ليست صعبة...,1
2,3,الحمد لله ارقام مقبولة مقارنة بدول المنطقة لول...,1
3,4,انا شخصيا أؤيد ما فرضته السلطات من ضرورة الإدل...,1
4,5,\nنفس الشئ في مدينة برشيد مراكز التلقيح مغلقة ...,1


In [ ]:
train.comment=train.comment.str.replace('\n|#','')

<ipython-input-5-4face38965c7>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train.comment=train.comment.str.replace('\n|#','')


In [ ]:
print(train['comment'][1])
#print(train['cleaned_text'][1])

X_data = train['comment']
y_data = np.array(train['label'])


هناك الكثير لا يفهم المقصود كورونا ليست صعبة على الشباب لكنها صعبة على اصحاب مرض مزمن كالسكري …. لهذا الدولة اختارت ان تنقد امهاتنا و ابائنا و جميع مرضى السكري و من بعد ذلك قام قبل سيعم الجميع هو في الحقيقة عندما يتم التلقيح و يتم محاصرة الفيروس بالتلقيح ينتهي المشكل قبل أن يعم الفيروس كل الشرائح لهذا نتمنى ان تسرع الدولة في مخططها هذا لما له من تأثير على محاربة الوباء في اقرب وقت ممكن نتمنى من الله. التوفيق لكل من يساهم من بعيد او قريب على انجاح هذه المهمة و الشكر كذلك لصاحب الجلالة الذي لا يبخل في ان يكون هو السابق لمثل هذه المبادرة التي ستعود عليه بالصحة و العافية و تحية للشعب المغربي من طنجة إلى الݣرݣرات


# <b>Fine Tune a bert model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")
bert_model = AutoModelForSequenceClassification.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
from sklearn.model_selection import train_test_split
X = X_data.tolist()
y = y_data.tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)


In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00


In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

# # Define Trainer
# args = TrainingArguments(
#     output_dir="Fine-tuned-msa-bert",
#     evaluation_strategy="steps",
#     eval_steps=500,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,
#     seed=0,
#     load_best_model_at_end=True,
# )
args = TrainingArguments(output_dir="Fine-tuned-msa-bert", evaluation_strategy="epoch")

trainer = Trainer(
    model=bert_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [ ]:
# Train pre-trained model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.451765,0.796875
2,No log,0.601278,0.830729
3,0.345200,0.831774,0.828125


TrainOutput(global_step=576, training_loss=0.30934619572427535, metrics={'train_runtime': 255.2776, 'train_samples_per_second': 18.051, 'train_steps_per_second': 2.256, 'total_flos': 653573729636352.0, 'train_loss': 0.30934619572427535, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/drive/My Drive/fineTunedBertModel_MSA')

# <b>use the Fined tuned Model</b>

In [ ]:
from transformers import AutoModel
Ft_model = AutoModel.from_pretrained("/content/drive/My Drive/fineTunedBertModel_MSA")

In [ ]:
texts = X_data.tolist()
labels = y_data.tolist()

tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### extract embedding

In [ ]:
def sent_to_embd(sentences, model, tokenizer):
    embd = list()
    for sentence in sentences:
        input_ids = tokenizer.encode(sentence, add_special_tokens=True)
        input_ids = torch.tensor([input_ids])
        with torch.no_grad():
            last_hidden_states = model(input_ids)[0]
        last_hidden_states = last_hidden_states.mean(1)
        embd.append(last_hidden_states)
    return np.vstack(embd)

In [ ]:
aggregated_embeddings = sent_to_embd(train.comment, Ft_model, tokenizer)
aggregated_embeddings.shape

(1920, 768)

### pass the embedding to Machine Learning Models

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(aggregated_embeddings, labels, test_size=0.1, random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier


models = {"Logistic Regression": LogisticRegression(),
          "KNN": KNeighborsClassifier(6),
          "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=4, random_state=42),
          "SVC": SVC(kernel = 'rbf'),
          "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, max_depth=1, random_state=0),
          "MLP Classifier": MLPClassifier(hidden_layer_sizes=(64,), max_iter=1000, random_state=42)
         }

def fit_and_score(models, X_train, X_test, y_train, y_test):
    np.random.seed(42)
    model_scores = {}
    best_model = None
    best_score = 0
    model_name = None

    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
        if model_scores[name] >= best_score:
          best_score = model_scores[name]
          best_model = model
          model_name = name
    return model_scores, best_model, best_score, model_name

In [ ]:
model_scores, model, score, name = fit_and_score(models=models,
                             X_train=X_train,
                             X_test=X_test,
                             y_train=y_train,
                             y_test=y_test)
print(model_scores)
print("model : ", name, " score : ",score)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Logistic Regression': 0.9635416666666666, 'KNN': 0.953125, 'Random Forest': 0.9635416666666666, 'SVC': 0.9583333333333334, 'Gradient Boosting': 0.9583333333333334, 'MLP Classifier': 0.9583333333333334}
model :  Random Forest  score :  0.9635416666666666


In [ ]:
filename = "/content/drive/My Drive/Models/Arabic_Bert_Model/model_version2.pickle"

# save model
pickle.dump(model, open(filename, "wb"))

# <b>Load Model

In [ ]:
filename = "/content/drive/My Drive/Models/Arabic_Bert_Model/model_version2.pickle"

loaded_model = pickle.load(open(filename, "rb"))


In [ ]:
#y_predicted = loaded_model.predict(X)
loaded_model.score(X_test, y_test)

0.9583333333333334

# Submission

In [ ]:
test_stage = pd.read_csv('/content/drive/My Drive/Datasets/marocSA/test_stage1.csv')

sample_submission = pd.read_csv('/content/drive/My Drive/Datasets/marocSA/sample_submission.csv')

In [ ]:
sample_submission.head()

,ID,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
test_stage.comment=test_stage.comment.str.replace('\n|#','')
submission = test_stage['comment']

submission_embeddings = sent_to_embd(submission, Ft_model, tokenizer)
submission_embeddings.shape

<ipython-input-32-74651a97a178>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_stage.comment=test_stage.comment.str.replace('\n|#','')


(240, 768)

In [ ]:
predictions = loaded_model.predict(submission_embeddings)
sample_submission['label'] = predictions

In [ ]:
sample_submission

,ID,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,0
...,...,...
235,236,0
236,237,0
237,238,1
238,239,0


In [ ]:
sample_submission.to_csv("/content/drive/My Drive/Models/Arabic_Bert_Model/submission.csv", index=False)